In [2]:
import numpy as np
import pandas as pd

In [3]:
ds_train = pd.read_parquet('../../data/otto/train.parquet')
ds_test = pd.read_parquet('../../data/otto/test.parquet')

In [4]:
ds_train = pd.concat((ds_train, ds_test), axis=0)
del ds_test

In [1]:
ds_train

NameError: name 'ds_train' is not defined

In [4]:
max_session_type = ds_train.groupby(['session', 'aid'])['type'].max().rename('max_type')

In [ ]:
ds_train = ds_train.join(max_session_type, ['session', 'aid'])

In [ ]:
ds_train = ds_train[ds_train['type'] == ds_train['max_type']]

In [ ]:
ds_train.reset_index(drop=True, inplace=True)
ds_train.drop(columns='max_type', inplace=True)

In [ ]:
click_weight, cart_weight, purchase_weight = \
    ds_train.shape[0] / ds_train['type'].value_counts()

In [ ]:
ds_train = ds_train.join(
    ds_train.groupby(['session', 'aid'])['type'].count().rename('count'),
    on=['session', 'aid']
)

In [ ]:
ds_train.drop_duplicates(['session', 'aid'], inplace=True)

In [ ]:
click_border = click_weight * (cart_weight // click_weight)
cart_border = cart_weight * (purchase_weight // cart_weight)

In [ ]:
ds_train['weight'] = np.where(
    ds_train['type'] == 0,
    np.minimum(click_border, ds_train['count'] * click_weight),
    np.where(
        ds_train['type'] == 1,
        np.minimum(cart_border, ds_train['count'] * cart_weight),
        ds_train['count'] * purchase_weight
    )
)

In [ ]:
ds_train.drop(columns=['type', 'count'], inplace=True)

In [ ]:
ds_train.to_parquet('../../data/otto/interactions.parquet')